# Enterprise Feature Store - FeatureProcess

## Disclaimer
The sample code (“Sample Code”) provided is not covered by any Teradata agreements. Please be aware that Teradata has no control over the model responses to such sample code and such response may vary. The use of the model by Teradata is strictly for demonstration purposes and does not constitute any form of certification or endorsement. The sample code is provided “AS IS” and any express or implied warranties, including the implied warranties of merchantability and fitness for a particular purpose, are disclaimed. In no event shall Teradata be liable for any direct, indirect, incidental, special, exemplary, or consequential damages (including, but not limited to, procurement of substitute goods or services; loss of use, data, or profits; or business interruption) sustained by you or a third party, however caused and on any theory of liability, whether in contract, strict liability, or tort arising in any way out of the use of this sample code, even if advised of the possibility of such damage.

## Context
**Sales Analytics Feature Store Demo**

This notebook demonstrates how to build and manage a feature store for sales analytics using TeradataML. It covers the end-to-end process of feature engineering, transformation, and governance for sales transaction data. The workflow includes:
- Loading and transforming raw sales data into engineered features such as transaction counts, maximum and total sales amounts, and region-based aggregations.
- Creating and managing a centralized feature store `analytics` within the `sales_transactions` data domain.
- Ingesting features like count, max, and total sales per region, as well as new and updated features for iterative analytics.
- Demonstrating feature lineage, versioning, and time-travel for reproducible machine learning workflows.
- Enabling collaboration and reusability of high-quality, governed features for downstream ML models and analytics.
The notebook focuses on features such as aggregated sales metrics, region-based statistics, and temporal feature updates, providing a robust foundation for enterprise-scale sales analytics and machine learning.

## 1. Load the required libraries

In [2]:
import os 
from teradataml import create_context, remove_context, DataFrame, FeatureProcess, FeatureStore, FeatureGroup, DataSource, Entity, Feature, \
load_example_data, in_schema, DatasetCatalog, db_drop_table, db_drop_view, FeatureCatalog, read_csv
from getpass import getpass
from collections import OrderedDict
from teradatasqlalchemy import INTEGER, FLOAT, VARCHAR

## 2. Connect to Vantage with Admin user 

Connecting to Vantage with an Admin user is required for initial setup tasks such as creating the feature store, configuring storage, and granting permissions to other users. These operations typically require elevated privileges.

In [6]:
context=create_context(config_file='admin_config_file.env')

## 3. Create FeatureStore

### 3.1. Create the Feature Store

In [9]:
fs = FeatureStore(repo="analytics", data_domain="sales_transactions")

Repo analytics does not exist. Run FeatureStore.setup() to create the repo and setup FeatureStore.


### 3.2 Setup the Feature Store

In [12]:
fs.setup()

True

### 3.3. Grant the access to user 

 **Note:** 
Granting read/write access to a user is necessary so they can create, modify, and manage features and metadata within the feature store. This ensures the specified user has the required permissions to work with the feature store objects. If needed, you can later revoke these rights using `fs.revoke.read_write(username)`.

In [15]:
username = getpass(prompt = 'username: ')
fs.grant.read_write(username)

username:  ········


True

## 4. Connect to Vantage with non-admin user

### 4.1. Remove context with Admin user

In [18]:
remove_context()

True

### 4.2 Create context with non-admin user

In [20]:
context=create_context(config_file='non_admin_config_file.env')

### 4.3. Create Feature Store object with non-admin user to ingest feature values. 

In [22]:
fs = FeatureStore(repo="analytics", data_domain="sales_transactions")

FeatureStore is ready to use.


## 5. Get Data For this demo

### 5.1.  Load the sales Data for first current week.

In [29]:
sales_dt = OrderedDict(CustomerID=INTEGER, Quantity=INTEGER, TotalAmount=INTEGER, Region=VARCHAR(20))
df1 = read_csv(table_name="week1_sales",
               filepath=r"../data/sales.csv",
               types=sales_dt)
df1.head(3)

CustomerID,Quantity,TotalAmount,Region
1004,4,55,South
1002,1,15,East
1001,3,162,North


### 5.2. Perform Data Transformation

**Transformation Details:**
In this step, we group the sales data by the 'Region' column and compute three key aggregated features for each region:
- `total_TotalAmount`: The sum of all sales amounts in the region.
- `count_TotalAmount`: The total number of sales transactions in the region.
- `max_TotalAmount`: The maximum sales amount for a single transaction in the region.
These features are essential for region-level analytics and are used as managed features in the feature store.

In [31]:
df1 = df1.groupby('Region').assign(total_TotalAmount=df1.TotalAmount.sum(),
                                   count_TotalAmount=df1.TotalAmount.count(),
                                   max_TotalAmount=df1.TotalAmount.max())
df1

Region,count_TotalAmount,max_TotalAmount,total_TotalAmount
East,2,776,791
South,3,237,494
West,2,480,867
North,3,428,673


### 5.3. Load the sales Data for week 2.

In [34]:
sales_new_dt = OrderedDict(CustomerID=INTEGER, Quantity=INTEGER, TotalAmount=INTEGER, Region=VARCHAR(20))
df2 = read_csv(table_name="week2_sales", 
               filepath=r"../data/sales_week2.csv", 
               types=sales_new_dt)
df2.head(3)

CustomerID,Quantity,TotalAmount,Region
1004,6,155,South
1002,3,115,East
1001,5,262,North


### 5.4. Perform Data Transformation

**Transformation Explanation:**

In this step, we perform the following transformations on the sales data for week 2:

1. **Group by 'Region'**: The data is grouped by the 'Region' column to enable region-level aggregations.
2. **Aggregate Features**:
   - `total_TotalAmount`: Calculates the sum of the `TotalAmount` for each region, representing the total sales amount per region.
   - `count_TotalAmount`: Counts the number of sales transactions for each region.
   - `max_TotalAmount`: Finds the maximum `TotalAmount` for a single transaction in each region.
3. **Tentative Feature Engineering**:
   - `tentative_incr_in_max_TotalAmount`: Adds 100 to the `max_TotalAmount` for each region, simulating a scenario where the maximum sales amount is projected to increase.
   - `tentative_incr_in_total_TotalAmount`: Adds 150 to the `total_TotalAmount` for each region, simulating a projected increase in total sales.

These engineered features are useful for advanced analytics, scenario planning, and as managed features in the feature store.

In [37]:
df2 = df2.groupby('Region').assign(total_TotalAmount=df2.TotalAmount.sum(),
                                   count_TotalAmount=df2.TotalAmount.count(),
                                   max_TotalAmount=df2.TotalAmount.max())
df2 = df2.assign(tentative_incr_in_max_TotalAmount=df2.max_TotalAmount+100, 
                tentative_incr_in_total_TotalAmount=df2.total_TotalAmount+150)
df2

Region,count_TotalAmount,max_TotalAmount,total_TotalAmount,tentative_incr_in_max_TotalAmount,tentative_incr_in_total_TotalAmount
West,2,580,1067,680,1217
East,2,876,991,976,1141
South,3,337,794,437,944
North,3,528,973,628,1123


## 6. Store the data transformations

We are storing the transformation here. So, even if underlying data varies, the data transformation steps remain same.

In [40]:
sales_df1 = df1.create_view("week1_sales_view")
sales_df1

Region,count_TotalAmount,max_TotalAmount,total_TotalAmount
South,3,237,494
North,3,428,673
East,2,776,791
West,2,480,867


In [41]:
sales_df2 = df2.create_view("week2_sales_view")
sales_df2

Region,count_TotalAmount,max_TotalAmount,total_TotalAmount,tentative_incr_in_max_TotalAmount,tentative_incr_in_total_TotalAmount
West,2,580,1067,680,1217
East,2,876,991,976,1141
North,3,528,973,628,1123
South,3,337,794,437,944


## 7. Ingest the features for current week
- Store the values of count_TotalAmount, max_TotalAmount, total_TotalAmount for every region.
- Run the FeatureProcess

### 7.1. Create the FeatureProcess and run it

In [44]:
fp1 = FeatureProcess(
    repo="analytics",
    data_domain="sales_transactions",
    object=sales_df1,
    entity="Region",
    features=["count_TotalAmount", "max_TotalAmount", "total_TotalAmount"]
)
fp1.run()

Process 'c2fabc23-8a28-11f0-82de-b0dcef8381ea' started.
Process 'c2fabc23-8a28-11f0-82de-b0dcef8381ea' completed.


True

### 7.2. See the mind_map for Feature Store

We ingested three features—`count_TotalAmount`, `max_TotalAmount`, and `total_TotalAmount`—from a single feature process. This demonstrates how multiple related features can be managed and tracked together within the feature store, maintaining their lineage to the originating process.

In [47]:
fs.mind_map()

### 7.3. Explore the Properties 

#### 7.3.1. process_id

The `process_id` property provides a unique identifier for this feature process run.

In [51]:
fp1.process_id

'c2fabc23-8a28-11f0-82de-b0dcef8381ea'

#### 7.3.2. entity

The `entity` property indicates the primary key or business entity for which the features were ingested. 

In [54]:
fp1.entity

Entity(name=Region)

#### 7.3.3. features

The `features` property lists all the features that were ingested by this feature process.

In [57]:
fp1.features

[Feature(name=count_TotalAmount),
 Feature(name=max_TotalAmount),
 Feature(name=total_TotalAmount)]

#### 7.3.4. status

The `status` property displays the current execution state of the feature process (e.g., running, completed, failed). 

In [60]:
fp1.status

'completed'

### 7.4. Let's cross verify week1 ingested data.

**Note**: To build real datasets on the ingested features, users should refer to `DatasetCatalog.build_dataset()`. The same applies to time series datasets as well.

In [63]:
df_fp1 = fs.get_data(process_id=fp1.process_id)
df_fp1

Region,count_TotalAmount,max_TotalAmount,total_TotalAmount
South,3,237,494
North,3,428,673
East,2,776,791
West,2,480,867


## 8. Ingest the features for week2

### 8.1. Create the FeatureProcess and run it for only North_week2 Region
- Store the values of count_TotalAmount, max_TotalAmount, total_TotalAmount, new_count_TotalAmount, new_max_TotalAmount
- Run it for every North_week2 region.

In [75]:
fp2 = FeatureProcess(
    repo="analytics",
    data_domain="sales_transactions",
    object=sales_df2,
    entity="Region",
    features=["count_TotalAmount", "max_TotalAmount", "total_TotalAmount", 
              "tentative_incr_in_max_TotalAmount", "tentative_incr_in_total_TotalAmount"]
)
fp2.run(filters=[f"Region='North'"])

Process 'ff360513-8a28-11f0-8266-b0dcef8381ea' started.
Ingesting the features for filter 'Region='North'' to catalog.
Process 'ff360513-8a28-11f0-8266-b0dcef8381ea' completed.


True

### 8.2. See the mind_map for Feature Store

In [77]:
fs.mind_map()

### 8.3. Let's cross verify week2 ingested data for only North region

**Note**: To build real datasets on the ingested features, users should refer to `DatasetCatalog.build_dataset()`. The same applies to time series datasets as well.

In [80]:
df_fp2 = fs.get_data(process_id=fp2.process_id)
df_fp2

Region,count_TotalAmount,max_TotalAmount,tentative_incr_in_max_TotalAmount,tentative_incr_in_total_TotalAmount,total_TotalAmount
North,3,528,628,1123,973


## 9. Ingest the features at specific time

### 9.1. Get the Feature Version table 

#### 9.1.1. Create FeatureCatalog Object

In [84]:
fc = FeatureCatalog(repo="analytics",
                    data_domain="sales_transactions")
fc

FeatureCatalog(repo=analytics, data_domain=sales_transactions)

#### 9.1.2. Check the feature versions

In [87]:
f_ver = fc.list_feature_versions()
f_ver

entity_id,data_domain,id,name,table_name,feature_version
Region,sales_transactions,2,max_TotalAmount,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,c2fabc23-8a28-11f0-82de-b0dcef8381ea
Region,sales_transactions,1,count_TotalAmount,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,c2fabc23-8a28-11f0-82de-b0dcef8381ea
Region,sales_transactions,1,count_TotalAmount,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,ff360513-8a28-11f0-8266-b0dcef8381ea
Region,sales_transactions,5,tentative_incr_in_total_TotalAmount,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,ff360513-8a28-11f0-8266-b0dcef8381ea
Region,sales_transactions,4,tentative_incr_in_max_TotalAmount,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,ff360513-8a28-11f0-8266-b0dcef8381ea
Region,sales_transactions,2,max_TotalAmount,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,ff360513-8a28-11f0-8266-b0dcef8381ea
Region,sales_transactions,3,total_TotalAmount,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,c2fabc23-8a28-11f0-82de-b0dcef8381ea
Region,sales_transactions,3,total_TotalAmount,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,ff360513-8a28-11f0-8266-b0dcef8381ea


#### 9.1.3. Get the data from feature table 

In [89]:
feat_df = DataFrame(in_schema('analytics', 'FS_T_74f39696_8ca1_2744_5460_b3d357904b4c'))
feat_df

Region,feature_id,feature_value,feature_version,valid_start,valid_end,ValidPeriod
West,3,867,c2fabc23-8a28-11f0-82de-b0dcef8381ea,2025-09-05 07:20:23.760000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:20:23.760000+0
South,1,3,c2fabc23-8a28-11f0-82de-b0dcef8381ea,2025-09-05 07:20:23.760000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:20:23.760000+0
North,1,3,c2fabc23-8a28-11f0-82de-b0dcef8381ea,2025-09-05 07:20:23.760000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:20:23.760000+0
North,1,3,ff360513-8a28-11f0-8266-b0dcef8381ea,2025-09-05 07:22:05.780000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:22:05.780000+0
South,2,237,c2fabc23-8a28-11f0-82de-b0dcef8381ea,2025-09-05 07:20:23.760000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:20:23.760000+0
North,2,428,c2fabc23-8a28-11f0-82de-b0dcef8381ea,2025-09-05 07:20:23.760000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:20:23.760000+0
North,2,528,ff360513-8a28-11f0-8266-b0dcef8381ea,2025-09-05 07:22:05.780000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:22:05.780000+0
East,3,791,c2fabc23-8a28-11f0-82de-b0dcef8381ea,2025-09-05 07:20:23.760000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:20:23.760000+0
South,3,494,c2fabc23-8a28-11f0-82de-b0dcef8381ea,2025-09-05 07:20:23.760000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:20:23.760000+0
East,2,776,c2fabc23-8a28-11f0-82de-b0dcef8381ea,2025-09-05 07:20:23.760000+00:,9999-12-31 23:59:59.999999+00:,('2025-09-05 07:20:23.760000+0


## 9.2. Ingest the features at specific time
- Store the values from FeatureProcess2 at specific time

In [92]:
fp3 = FeatureProcess(
    repo="analytics",
    data_domain="sales_transactions",
    object=fp2.process_id
)
fp3.run(as_of='2025-09-05 07:21:59.000000+00:00')

Process 'ff360513-8a28-11f0-8266-b0dcef8381ea' started.
Process 'ff360513-8a28-11f0-8266-b0dcef8381ea' completed.


True

## 9.3 See the mind_map for Feature Store

In [93]:
fs.mind_map()

### 9.4. Let's cross verify week1 ingested data

**Note**: To build real datasets on the ingested features, users should refer to `DatasetCatalog.build_dataset()`. The same applies to time series datasets as well.

In [96]:
df_fp3 = fs.get_data(process_id=fp3.process_id)
df_fp3

Region,count_TotalAmount,max_TotalAmount,tentative_incr_in_max_TotalAmount,tentative_incr_in_total_TotalAmount,total_TotalAmount
North,3,528,628,1123,973
West,2,580,680,1217,1067
South,3,337,437,944,794
East,2,876,976,1141,991


## 10. Explore the FeatureCatalog

### 10.1. Explore FeatureCatalog Properties

#### 10.1.1. features

The `features` property of the feature catalog lists all features currently registered in the catalog.

In [99]:
fc.features

[Feature(name=count_TotalAmount),
 Feature(name=tentative_incr_in_max_TotalAmount),
 Feature(name=max_TotalAmount),
 Feature(name=total_TotalAmount),
 Feature(name=tentative_incr_in_total_TotalAmount)]

#### 10.1.2. entities

The `entities` property lists all entities (primary keys or business keys) managed in the feature catalog.

In [102]:
fc.entities

[Entity(name=Region)]

#### 10.1.3. data_domain

The `data_domain` property shows the business domain associated with the feature catalog.

In [105]:
fc.data_domain

'sales_transactions'

### 10.2. Explore FeatureCatalog Methods

#### 10.2.1. List Features

In [109]:
fc.list_features()

feature_id,name,entity_name,data_type,feature_type,valid_start,valid_end
1,count_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:20:20.650000+00:,9999-12-31 23:59:59.999999+00:
4,tentative_incr_in_max_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:22:02.250000+00:,9999-12-31 23:59:59.999999+00:
2,max_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:20:20.650000+00:,9999-12-31 23:59:59.999999+00:
3,total_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:20:20.650000+00:,9999-12-31 23:59:59.999999+00:
5,tentative_incr_in_total_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:22:02.250000+00:,9999-12-31 23:59:59.999999+00:


#### 10.2.2. Archive Features

In [111]:
fc.archive_features(features='total_TotalAmount')

Feature 'total_TotalAmount' is archived from table 'FS_T_74f39696_8ca1_2744_5460_b3d357904b4c'.
Feature 'total_TotalAmount' is archived from metadata.


True

In [112]:
fc.list_features()

feature_id,name,entity_name,data_type,feature_type,valid_start,valid_end
3,total_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:20:20.650000+00:,2025-09-05 07:24:29.060000+00:
2,max_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:20:20.650000+00:,9999-12-31 23:59:59.999999+00:
1,count_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:20:20.650000+00:,9999-12-31 23:59:59.999999+00:
5,tentative_incr_in_total_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:22:02.250000+00:,9999-12-31 23:59:59.999999+00:
4,tentative_incr_in_max_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:22:02.250000+00:,9999-12-31 23:59:59.999999+00:


#### 10.2.3. Delete Features

In [114]:
fc.delete_features(features='total_TotalAmount')

Feature 'total_TotalAmount' is deleted from table 'FS_T_74f39696_8ca1_2744_5460_b3d357904b4c'.
Feature 'total_TotalAmount' is deleted from metadata.


True

In [115]:
fc.list_features()

feature_id,name,entity_name,data_type,feature_type,valid_start,valid_end
2,max_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:20:20.650000+00:,9999-12-31 23:59:59.999999+00:
4,tentative_incr_in_max_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:22:02.250000+00:,9999-12-31 23:59:59.999999+00:
1,count_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:20:20.650000+00:,9999-12-31 23:59:59.999999+00:
5,tentative_incr_in_total_TotalAmount,Region,INTEGER,CONTINUOUS,2025-09-05 07:22:02.250000+00:,9999-12-31 23:59:59.999999+00:


## 11. Explore the FeatureStore

### 11.1. List feature_catalogs

In [118]:
fs.list_feature_catalogs()

entity_name,data_domain,feature_id,table_name,valid_start,valid_end
Region,sales_transactions,5,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,2025-09-05 07:22:02.250000+00:,9999-12-31 23:59:59.999999+00:
Region,sales_transactions,4,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,2025-09-05 07:22:02.250000+00:,9999-12-31 23:59:59.999999+00:
Region,sales_transactions,1,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,2025-09-05 07:20:20.650000+00:,9999-12-31 23:59:59.999999+00:
Region,sales_transactions,2,FS_T_74f39696_8ca1_2744_5460_b3d357904b4c,2025-09-05 07:20:20.650000+00:,9999-12-31 23:59:59.999999+00:


### 11.2. List Entities

In [120]:
fs.list_entities()

name,data_domain,description,creation_time,modified_time,entity_column
Region,sales_transactions,None,2025-09-05 07:20:16.350404,None,Region


### 11.3. List feature_processes

In [122]:
fs.list_feature_processes()

process_id,description,data_domain,process_type,data_source,entity_id,feature_names,feature_ids,valid_start,valid_end
ff360513-8a28-11f0-8266-b0dcef8381ea,,sales_transactions,denormalized view,"""ALICE"".""week2_sales_view""",Region,"count_TotalAmount, max_TotalAmount, tentative_incr_in_max_TotalAmount, tentative_incr_in_total_TotalAmount, total_TotalAmount",None,2025-09-05 07:22:08.500000+00:,9999-12-31 23:59:59.999999+00:
c2fabc23-8a28-11f0-82de-b0dcef8381ea,,sales_transactions,denormalized view,"""ALICE"".""week1_sales_view""",Region,"count_TotalAmount, max_TotalAmount, total_TotalAmount",None,2025-09-05 07:20:25.690000+00:,9999-12-31 23:59:59.999999+00:


### 11.4. List feature_runs

In [124]:
fs.list_feature_runs()

run_id,process_id,data_domain,start_time,end_time,status,filter,as_of_start,as_of_end,failure_reason
2,ff360513-8a28-11f0-8266-b0dcef8381ea,sales_transactions,2025-09-05 07:21:56.620649,2025-09-05 07:22:08.217706,completed,Region='North',None,None,None
1,c2fabc23-8a28-11f0-82de-b0dcef8381ea,sales_transactions,2025-09-05 07:20:15.749483,2025-09-05 07:20:25.379254,completed,None,None,None,None
3,ff360513-8a28-11f0-8266-b0dcef8381ea,sales_transactions,2025-09-05 07:23:28.279066,2025-09-05 07:23:40.347898,completed,None,2025-09-05 07:21:59.000000+00:,9999-12-31 23:59:59.999999+00:,None


### 11.5. List Features

In [126]:
fs.list_features()

id,name,data_domain,column_name,description,tags,data_type,feature_type,status,creation_time,modified_time,group_name
5,tentative_incr_in_total_TotalAmount,sales_transactions,tentative_incr_in_total_TotalAmount,None,None,INTEGER,CONTINUOUS,ACTIVE,2025-09-05 07:22:01.340007,2025-09-05 07:23:32.159557,None
4,tentative_incr_in_max_TotalAmount,sales_transactions,tentative_incr_in_max_TotalAmount,None,None,INTEGER,CONTINUOUS,ACTIVE,2025-09-05 07:22:00.436747,2025-09-05 07:23:31.267610,None
2,max_TotalAmount,sales_transactions,max_TotalAmount,None,None,INTEGER,CONTINUOUS,ACTIVE,2025-09-05 07:20:18.778578,2025-09-05 07:23:30.363339,None
3,total_TotalAmount,sales_transactions,total_TotalAmount,None,None,INTEGER,CONTINUOUS,ACTIVE,2025-09-05 07:20:19.730569,2025-09-05 07:23:33.055104,None
1,count_TotalAmount,sales_transactions,count_TotalAmount,None,None,INTEGER,CONTINUOUS,ACTIVE,2025-09-05 07:20:17.847453,2025-09-05 07:23:29.451338,None


## 12. Cleanup 

### 12.1 Drop Views

In [129]:
db_drop_view('week1_sales_view')

True

In [130]:
db_drop_view('week2_sales_view')

True

### 12.2 Drop Tables

In [132]:
db_drop_table('week1_sales')

True

In [133]:
db_drop_table('week2_sales')

True

### 12.3 Remove Context

In [135]:
remove_context()

True

### 12.4. Delete the FeatureStore

In [161]:
context=create_context(config_file='admin_config_file.env')

**Note** : This will drop the database if all objects are removed.

In [142]:
fs = FeatureStore(repo="analytics", data_domain="sales_transactions")
fs.delete()

FeatureStore is ready to use.


The function removes Feature Store and drops the corresponding repo also. Are you sure you want to proceed? (Y/N):  y


True

In [163]:
remove_context()

True